# Order Confirmation and Notification System
Implementation of SMS and email notification services for EasyEats order management

In [ ]:
# Import required libraries
import os
import requests
from twilio.rest import Client
from sendgrid import SendGridAPIClient
from sendgrid.helpers.mail import Mail
import json
import logging
from dotenv import load_dotenv

## Configure Third-Party Services
Set up API keys and client configurations for Twilio (SMS) and SendGrid (Email)

In [ ]:
# Load environment variables
load_dotenv()

# Initialize Twilio client with error handling
try:
    TWILIO_ACCOUNT_SID = os.getenv('TWILIO_ACCOUNT_SID')
    TWILIO_AUTH_TOKEN = os.getenv('TWILIO_AUTH_TOKEN')
    TWILIO_PHONE_NUMBER = os.getenv('TWILIO_PHONE_NUMBER')
    
    if not all([TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN, TWILIO_PHONE_NUMBER]):
        raise ValueError('Missing required Twilio environment variables')
        
    twilio_client = Client(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN)
    print('Twilio client initialized successfully')
except Exception as e:
    print(f'Error initializing Twilio client: {str(e)}')
    twilio_client = None

# Initialize SendGrid client with error handling
try:
    SENDGRID_API_KEY = os.getenv('SENDGRID_API_KEY')
    
    if not SENDGRID_API_KEY:
        raise ValueError('Missing SendGrid API key')
        
    sendgrid_client = SendGridAPIClient(SENDGRID_API_KEY)
    print('SendGrid client initialized successfully')
except Exception as e:
    print(f'Error initializing SendGrid client: {str(e)}')
    sendgrid_client = None

## Customer Order Confirmation Functions
Implementation of SMS and email confirmation sending functions

In [ ]:
def send_order_confirmation_sms(customer_phone, order_details):
    if not twilio_client:
        logging.error('Twilio client not initialized')
        return False, 'Twilio client not initialized'
        
    try:
        if not customer_phone:
            raise ValueError('Customer phone number is required')
            
        message = twilio_client.messages.create(
            body=f"Your EasyEats order #{order_details.get('order_id', 'N/A')} has been confirmed! "
                 f"Estimated delivery time: {order_details.get('estimated_delivery_time', 'N/A')}",
            from_=TWILIO_PHONE_NUMBER,
            to=customer_phone
        )
        logging.info(f'SMS sent successfully: {message.sid}')
        return True, message.sid
    except Exception as e:
        error_msg = f'SMS sending failed: {str(e)}'
        logging.error(error_msg)
        return False, error_msg

def send_order_confirmation_email(customer_email, order_details):
    if not sendgrid_client:
        logging.error('SendGrid client not initialized')
        return False, 'SendGrid client not initialized'
        
    try:
        if not customer_email:
            raise ValueError('Customer email is required')
            
        message = Mail(
            from_email=os.getenv('SENDGRID_FROM_EMAIL', 'orders@easyeats.com'),
            to_emails=customer_email,
            subject=f'EasyEats Order Confirmation #{order_details.get("order_id", "N/A")}',
            html_content=f'''
                <h2>Thank you for your order!</h2>
                <p><strong>Your order has been confirmed!</strong></p>
                <p>Order ID: {order_details.get("order_id", "N/A")}</p>
                <p>Total Amount: ${order_details.get("total_amount", "0.00")}</p>
                <p>Estimated Delivery: {order_details.get("estimated_delivery_time", "N/A")}</p>
            '''
        )
        response = sendgrid_client.send(message)
        logging.info(f'Email sent successfully: {response.status_code}')
        return True, response.status_code
    except Exception as e:
        error_msg = f'Email sending failed: {str(e)}'
        logging.error(error_msg)
        return False, error_msg

## Delivery Personnel Notification System
Functions to notify delivery personnel about new orders

In [ ]:
def notify_delivery_personnel(delivery_person_contact, order_details):
    if not twilio_client:
        logging.error('Twilio client not initialized')
        return False, 'Twilio client not initialized'
        
    try:
        if not delivery_person_contact:
            raise ValueError('Delivery person contact is required')
            
        message = twilio_client.messages.create(
            body=f"New delivery assignment!\n"
                 f"Order #{order_details.get('order_id', 'N/A')}\n"
                 f"Pickup: {order_details.get('restaurant_address', 'N/A')}\n"
                 f"Delivery: {order_details.get('delivery_address', 'N/A')}",
            from_=TWILIO_PHONE_NUMBER,
            to=delivery_person_contact
        )
        logging.info(f'Delivery notification sent successfully: {message.sid}')
        return True, message.sid
    except Exception as e:
        error_msg = f'Delivery notification failed: {str(e)}'
        logging.error(error_msg)
        return False, error_msg

## Test Notification System
Test the implementation with sample order data

In [ ]:
# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# Sample order data for testing
test_order = {
    "order_id": "ORD123456",
    "customer_phone": "+1234567890",
    "customer_email": "customer@example.com",
    "total_amount": 45.90,
    "estimated_delivery_time": "30-45 minutes",
    "restaurant_address": "123 Restaurant St",
    "delivery_address": "456 Customer Ave",
    "delivery_person_contact": "+1987654321"
}

def run_tests(order_data):
    print('Starting notification tests...')
    
    # Test SMS notification
    print('\n1. Testing SMS notification:')
    sms_success, sms_result = send_order_confirmation_sms(
        order_data["customer_phone"], 
        order_data
    )
    print(f"Result: {'Success' if sms_success else 'Failed'} - {sms_result}")
    
    # Test email notification
    print('\n2. Testing email notification:')
    email_success, email_result = send_order_confirmation_email(
        order_data["customer_email"], 
        order_data
    )
    print(f"Result: {'Success' if email_success else 'Failed'} - {email_result}")
    
    # Test delivery notification
    print('\n3. Testing delivery personnel notification:')
    delivery_success, delivery_result = notify_delivery_personnel(
        order_data["delivery_person_contact"], 
        order_data
    )
    print(f"Result: {'Success' if delivery_success else 'Failed'} - {delivery_result}")

# Run the tests
run_tests(test_order)